# Resolution of the elastic problem of a sphere in an infinite matrix

In [ ]:
using TensND, LinearAlgebra, SymPy, Tensors, OMEinsum, Rotations
sympy.init_printing(use_unicode=true)

## Definition of the coordinate system, base vectors...

In [ ]:
Spherical = coorsys_spherical()
θ, ϕ, r = getcoords(Spherical) # Note the order of coordinates not `r, θ, ϕ` but `θ, ϕ, r` so that the frame `(𝐞ᶿ, 𝐞ᵠ, 𝐞ʳ)` coincides with the canonical frame for null angles
𝐞ᶿ, 𝐞ᵠ, 𝐞ʳ = unitvec(Spherical) # "\bfe<TAB>" to write bold `𝐞` and "\^\theta<TAB><TAB>" to write superscript `ᶿ`
𝐱 = getOM(Spherical) # Defines the current position vector in terms of spherical coordinates (ie `𝐱 = r 𝐞ʳ`)
𝐞₁, 𝐞₂, 𝐞₃ = unitvec(coorsys_cartesian()) ;

In [ ]:
𝟏, 𝟙, 𝕀, 𝕁, 𝕂 = init_isotropic() # Defines usual isotropic Tensors
k, μ = symbols("k μ", positive = true)
λ = k -2μ/3 ;

## General resolution

### Hydrostatic loading

\begin{equation}
\mathbf{u}\underset{||\mathbf{x}||\to\infty}{\sim}\mathbf{E}\cdot\mathbf{x} \textrm{ with } \mathbf{E}=\frac{1}{3}E_v\mathbf{1}
\end{equation}

The displacement field is naturally searched in a form satisfying the isotropy of the loading ie
\begin{equation}
\mathbf{u}=u_r(r)\,\mathbf{e}_r
\end{equation}


In [ ]:
u = SymFunction("u", real = true)
𝐮ˢᵖʰ = u(r) * 𝐞ʳ  # Note that the vector is in bold font ("\bfu<TAB>") and the component in normal font
𝛆ˢᵖʰ = simplify(SYMGRAD(𝐮ˢᵖʰ, Spherical)) # Strain tensor ("\bfepsilon<TAB>")

In [ ]:
𝛔ˢᵖʰ = simplify(λ * tr(𝛆ˢᵖʰ) * 𝟏 + 2μ * 𝛆ˢᵖʰ) # Stress tensor ("\bfsigma<TAB>")
𝐓ˢᵖʰ = simplify(𝛔ˢᵖʰ ⋅ 𝐞ʳ) ;

In [ ]:
div𝛔ˢᵖʰ = DIV(𝛔ˢᵖʰ, Spherical) ;

In [ ]:
eqˢᵖʰ = factor(simplify(div𝛔ˢᵖʰ ⋅ 𝐞ʳ))

In [ ]:
solˢᵖʰ = dsolve(eqˢᵖʰ, u(r)) ;

In [ ]:
ûˢᵖʰ = solˢᵖʰ.rhs() ; display(ûˢᵖʰ)
T̂ˢᵖʰ = factor(simplify(subs(𝐓ˢᵖʰ ⋅ 𝐞ʳ, u(r) => ûˢᵖʰ))) ; display(T̂ˢᵖʰ)

### Deviatoric loading

\begin{equation}
\mathbf{u}\underset{||\mathbf{x}||\to\infty}{\sim}\mathbf{E}\cdot\mathbf{x} \textrm{ with } \mathbf{E}=E\,(\mathbf{e}_1\otimes\mathbf{e}_1+\mathbf{e}_2\,\otimes\mathbf{e}_2-2\mathbf{e}_3\,\otimes\mathbf{e}_3)=\mathbb{1}-3\mathbf{e}_3\otimes\mathbf{e}_3
\end{equation}

Note that such a macroscopic strain tensor induces a symmetry of revolution of the fields, which means in particular that the displacement field is expect of the form

\begin{equation}
\mathbf{u}=u_\theta(\theta,r)\,\mathbf{e}_\theta+u_r(\theta,r)\,\mathbf{e}_r
\end{equation}

In [ ]:
𝐄 = 𝟏 - 3𝐞₃⊗𝐞₃
# Remote trends in θ of the displacement
fᶿ = simplify(𝐞ᶿ ⋅ 𝐄 ⋅ 𝐞ʳ)
fʳ = simplify(𝐞ʳ ⋅ 𝐄 ⋅ 𝐞ʳ) ;


In [ ]:
uᶿ = SymFunction("uᶿ", real = true)
uʳ = SymFunction("uʳ", real = true)
𝐮ᵈᵉᵛ = uᶿ(r)* fᶿ * 𝐞ᶿ + uʳ(r)* fʳ * 𝐞ʳ
𝛆ᵈᵉᵛ = simplify(SYMGRAD(𝐮ᵈᵉᵛ, Spherical)) ;

In [ ]:
𝛔ᵈᵉᵛ = simplify(λ * tr(𝛆ᵈᵉᵛ) * 𝟏 + 2μ * 𝛆ᵈᵉᵛ)
𝐓ᵈᵉᵛ = simplify(𝛔ᵈᵉᵛ ⋅ 𝐞ʳ) ;

In [ ]:
div𝛔ᵈᵉᵛ = DIV(𝛔ᵈᵉᵛ, Spherical) ;

In [ ]:
eqᶿᵈᵉᵛ = factor(simplify(div𝛔ᵈᵉᵛ ⋅ 𝐞ᶿ) / fᶿ) ;

In [ ]:
eqʳᵈᵉᵛ = factor(simplify(div𝛔ᵈᵉᵛ ⋅ 𝐞ʳ) / fʳ) ;

In [ ]:
α, Λ = symbols("α Λ", real = true)
eqᵈᵉᵛ = simplify.(subs.([eqᶿᵈᵉᵛ,eqʳᵈᵉᵛ], uᶿ(r) => r^α, uʳ(r) => Λ*r^α))

In [ ]:
αΛ = solve(eqᵈᵉᵛ, [α, Λ])

In [ ]:
ûᶿᵈᵉᵛ = sum([Sym("C$(i+2)") * r^αΛ[i][1] for i ∈ 1:length(αΛ)]) ; display(ûᶿᵈᵉᵛ)
ûʳᵈᵉᵛ = sum([Sym("C$(i+2)") * αΛ[i][2] * r^αΛ[i][1] for i ∈ 1:length(αΛ)]) ; display(ûʳᵈᵉᵛ)

In [ ]:
T̂ᶿᵈᵉᵛ = factor(simplify(subs(𝐓ᵈᵉᵛ ⋅ 𝐞ᶿ / fᶿ, uᶿ(r) => ûᶿᵈᵉᵛ, uʳ(r) => ûʳᵈᵉᵛ))) ; display(T̂ᶿᵈᵉᵛ)
T̂ʳᵈᵉᵛ = factor(simplify(subs(𝐓ᵈᵉᵛ ⋅ 𝐞ʳ / fʳ, uᶿ(r) => ûᶿᵈᵉᵛ, uʳ(r) => ûʳᵈᵉᵛ))) ; display(T̂ʳᵈᵉᵛ)
